# Gaussian Path Planning test

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Map config

map_name = "corridor"
num_samples = 500

# Agent config

num_agent = 500
agent_radius = 0.6
starts = [0, 1]
goals = [0, 1]
starts_weight = [0.5, 0.5]
goals_weight = [0.5, 0.5]


In [ ]:
import pickle
from swarm_prm.solvers.macro import SOLVER_REGISTRY 

solver_cls = SOLVER_REGISTRY["TEGSolver"]
map_fname = "maps/{}_{}.pkl".format(map_name, num_samples)

with open(map_fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_count = gaussian_prm.get_agent_assignment(num_agent, starts_weight)
goals_agent_count = gaussian_prm.get_agent_assignment(num_agent, goals_weight)

teg_graph= solver_cls(gaussian_prm, num_agent, agent_radius, 
                   starts, goals, starts_agent_count, goals_agent_count, 
                   None, max_time=10)
timestep, flow_dict, residual_graph = teg_graph.get_solution()

with open("solutions/flow_dict.pkl", "wb") as f:
    pickle.dump(flow_dict, f)

In [ ]:
import pickle

with open(map_fname, "rb") as f:
    gaussian_prm = pickle.load(f)

with open("solutions/flow_dict.pkl", "rb") as f:
    flow_dict = pickle.load(f)

macro_sol = gaussian_prm.get_macro_solution(flow_dict)
simple_paths, gaussian_paths = gaussian_prm.get_solution(flow_dict, timestep, num_agent)
with open("solutions/macro_sol.pkl", "wb") as f:
    pickle.dump(macro_sol, f)

In [ ]:
print(len(simple_paths))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = gaussian_prm.visualize_g_nodes()

# apf_solver.animate_solution(fig, ax)
for path in simple_paths:
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
#
plt.show()

## Gaussian Trajectory

In [ ]:
import pickle
from matplotlib import pyplot as plt
from swarm_prm.solvers.micro import GaussianTrajectorySolver

with open(map_fname, "rb") as f:
    gaussian_prm = pickle.load(f)

with open("solutions/macro_sol.pkl", "rb") as f:
    macro_sol = pickle.load(f)

timestep = max(macro_sol.keys())

gt_solver = GaussianTrajectorySolver(gaussian_prm, macro_sol, timestep, num_agent, starts_agent_count, goals_agent_count, safety_gap=0.2)
gt_paths = gt_solver.solve()

fig, ax = gaussian_prm.visualize_roadmap()

for path in simple_paths:
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-*', label='Path', color='blue')
# 
cmap = plt.get_cmap("tab10")

for i, path in enumerate(gt_paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '--', label='Path', color=cmap(i%10))


In [ ]:
plt.show()

In [ ]:
# # Add interpolations 
# 
# import numpy as np
# 
# def add_linear_interpolation_points(path, subdiv=5):
#     """
#     Insert linearly-interpolated positions between consecutive points.
#     
#     Parameters
#     ----------
#     path : array-like of shape (N, 2)
#         The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
#     subdiv : int
#         Number of intervals to subdivide each segment. 
#         For each original segment, we will add 'subdiv - 1' new interior points 
#         (except that we skip duplicates at the shared boundary).
# 
#     Returns
#     -------
#     new_path : np.ndarray of shape (M, 2)
#         The new path including the original points and the added interpolation points.
#     """
#     path = np.asarray(path)
#     new_path = []
# 
#     # Iterate over pairs of consecutive points
#     for i in range(len(path) - 1):
#         start = path[i]
#         end = path[i + 1]
#         
#         # Create subdiv+1 points in [start, end] using linspace for each dimension
#         xs = np.linspace(start[0], end[0], subdiv + 1)
#         ys = np.linspace(start[1], end[1], subdiv + 1)
#         
#         # Add each intermediate point except the very last one
#         # to avoid duplicating the next segment's start
#         for j in range(subdiv):
#             new_path.append([xs[j], ys[j]])
# 
#     # Finally add the very last point of the last segment
#     new_path.append(path[-1].tolist())
#     
#     return np.array(new_path)
# 
# interpolated_paths = []
# for path in gt_paths:
#     new_path = add_linear_interpolation_points(path, 50)
#     interpolated_paths.append(new_path)
# 

In [ ]:
# def animate_solution(fig, ax, speed, paths):
#     """
#         Visualize solution trajectory provided instance
#     """
#     
#     agents = []
#     def init():
#         for agent in agents:
#             agent.remove()
#         agents.clear()
#         return []
# 
#     def update(frame):
#         idx = frame * speed
#         for agent in agents:
#             agent.remove()
#         agents.clear()
#         cmap = plt.get_cmap("tab10")
#         locs = [path[idx] for path in paths]
#         for i, loc in enumerate(locs):
#             agent = ax.add_patch(Circle(loc, radius=agent_radius, color=cmap(i%10)))
#             agents.append(agent)
#         return agents
# 
#     anim = FuncAnimation(fig, update, frames=len(paths[0])// speed , 
#                          init_func=init, blit=True, interval=10)
#     anim.save("solutions/apf_solution.gif", writer='pillow', fps=6)
# 
# # fig, ax = instance.visualize()
# fig, ax = gaussian_prm.visualize_g_nodes()
# animate_solution(fig, ax, 10, interpolated_paths)